In [54]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import sqlite3
import sys, traceback
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import torch.nn as nn

In [55]:
def get_data_from_consumption(db_path, table_name):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print("successfully connected to database")
            
    except:
        print("Error in connecting database!")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)
        print("-"*60)
    df = pd.read_sql('SELECT * FROM {}'.format(table_name), conn)
    conn.close()
    return df

In [56]:
df = get_data_from_consumption("../../database/data.db", "CONSUMPTION")
df.head()

successfully connected to database


,unique_data,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,RawTimeStamp,ConsumptionTimeStamp
0,2549.026455,842,0,2.2,0,1,0,7,0.6,188,...,2549,9,7,19,0,0,1,1,2023-05-29 12:46:43,2023-05-29 14:32:00
1,2631.455231,1021,1,0.5,1,0,1,53,0.7,136,...,2631,17,3,7,1,1,0,2,2023-05-29 12:46:43,2023-05-29 14:32:00
2,2603.736014,563,1,0.5,1,2,1,41,0.9,145,...,2603,11,2,9,1,1,0,2,2023-05-29 12:46:43,2023-05-29 14:32:00
3,2769.680851,615,1,2.5,0,0,0,10,0.8,131,...,2769,16,8,11,1,0,0,2,2023-05-29 12:46:43,2023-05-29 14:32:00
4,1411.996700,1821,1,1.2,0,13,1,44,0.6,141,...,1411,8,2,15,1,1,0,1,2023-05-29 12:46:43,2023-05-29 14:32:00


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   unique_data           2000 non-null   float64
 1   battery_power         2000 non-null   int64  
 2   blue                  2000 non-null   int64  
 3   clock_speed           2000 non-null   float64
 4   dual_sim              2000 non-null   int64  
 5   fc                    2000 non-null   int64  
 6   four_g                2000 non-null   int64  
 7   int_memory            2000 non-null   int64  
 8   m_dep                 2000 non-null   float64
 9   mobile_wt             2000 non-null   int64  
 10  n_cores               2000 non-null   int64  
 11  pc                    2000 non-null   int64  
 12  px_height             2000 non-null   int64  
 13  px_width              2000 non-null   int64  
 14  ram                   2000 non-null   int64  
 15  sc_h                 

In [58]:
X = df.drop(["unique_data", "RawTimeStamp", "ConsumptionTimeStamp", "price_range"], axis=1)
y_train = df.iloc[:,-3].values
print(X.shape, y_train.shape)


(2000, 20) (2000,)


In [59]:
scaler = Normalizer()
X_train = scaler.fit_transform(X)

In [60]:
class Data(Dataset):
    def __init__(self, X_train, y_train):
        self.X = torch.from_numpy(X_train.astype(np.float32))
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len

In [61]:
traindata = Data(X_train, y_train)

In [62]:
print(traindata[25:34])

(tensor([[2.9214e-01, 3.0400e-04, 4.2560e-04, 3.0400e-04, 0.0000e+00, 3.0400e-04,
         1.7328e-02, 1.8240e-04, 3.4656e-02, 2.4320e-03, 9.1199e-04, 8.8463e-02,
         4.3593e-01, 8.4572e-01, 5.4719e-03, 2.7360e-03, 2.1280e-03, 3.0400e-04,
         3.0400e-04, 3.0400e-04],
        [1.3326e-01, 2.5676e-04, 4.1081e-04, 2.5676e-04, 1.7973e-03, 2.5676e-04,
         1.3095e-02, 7.7028e-05, 3.3892e-02, 1.0270e-03, 4.8784e-03, 1.4122e-01,
         1.6561e-01, 9.6618e-01, 4.1081e-03, 2.5676e-04, 1.0270e-03, 2.5676e-04,
         0.0000e+00, 2.5676e-04],
        [2.6362e-01, 0.0000e+00, 1.3788e-04, 0.0000e+00, 2.7575e-04, 2.7575e-04,
         1.1306e-02, 2.7575e-04, 3.9432e-02, 1.9303e-03, 1.6545e-03, 1.4091e-01,
         2.9643e-01, 9.0612e-01, 4.6878e-03, 2.2060e-03, 3.3090e-03, 2.7575e-04,
         2.7575e-04, 0.0000e+00],
        [4.7120e-01, 0.0000e+00, 5.1887e-04, 3.2430e-04, 3.8915e-03, 3.2430e-04,
         1.6863e-02, 9.7289e-05, 3.1132e-02, 6.4859e-04, 5.8373e-03, 6.0643e-02,
      

In [63]:
batch_size = 4
trainloader = DataLoader(traindata, batch_size=batch_size, shuffle=True)

In [64]:
input_dim = 20
hidden_layers = 25
output_dim = 4

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear1 = nn.ReLU(input_dim, hidden_layers)
        self.linear2 = nn.Softmax(hidden_layers, output_dim)
    
    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [65]:
clf = Network()

TypeError: ReLU.__init__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
print(clf.parameters())

<generator object Module.parameters at 0x00000174E274ACE0>


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clf.parameters(), lr=0.1)

In [ ]:
epochs = 500
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        input, labels = data
        optimizer.zero_grad()
        outputs = clf(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss = running_loss + loss.item()
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

[1,   500] loss: 0.27244
[2,   500] loss: 0.21557
[3,   500] loss: 0.19430
[4,   500] loss: 0.18478
[5,   500] loss: 0.18463
[6,   500] loss: 0.17728
[7,   500] loss: 0.17804
[8,   500] loss: 0.17659
[9,   500] loss: 0.17732
[10,   500] loss: 0.17021
[11,   500] loss: 0.17044
[12,   500] loss: 0.17359
[13,   500] loss: 0.16884
[14,   500] loss: 0.16745
[15,   500] loss: 0.17201
[16,   500] loss: 0.16850
[17,   500] loss: 0.16738
[18,   500] loss: 0.16501
[19,   500] loss: 0.16852
[20,   500] loss: 0.16503
[21,   500] loss: 0.16362
[22,   500] loss: 0.16599
[23,   500] loss: 0.16295
[24,   500] loss: 0.16299
[25,   500] loss: 0.16106
[26,   500] loss: 0.16584
[27,   500] loss: 0.15880
[28,   500] loss: 0.16300
[29,   500] loss: 0.15913
[30,   500] loss: 0.16248
[31,   500] loss: 0.16030
[32,   500] loss: 0.16127
[33,   500] loss: 0.15717
[34,   500] loss: 0.15808
[35,   500] loss: 0.15610
[36,   500] loss: 0.16036
[37,   500] loss: 0.16135
[38,   500] loss: 0.15651
[39,   500] loss: 0.1